In [1]:
import os
import urllib.request
from io import BytesIO
import requests
import json

import torch

from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy
import h5py

import open3d as o3d

import matplotlib.image as mpimg
import re
from csv import writer

from utils.preprocessing import *
from models.model_data import *
from models_3d import point_clouds
from models import controlnet_model_wrapper



WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 1.13.1+cu117 with CUDA 1107 (you have 1.12.1+cpu)
    Python  3.8.10 (you have 3.8.16)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
C:\Users\User\anaconda3\envs\stable_diff\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\User\anaconda3\envs\stable_diff\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
nyu_path = 'C:/Users/User/Documents/Data_Science/ar_stable_diffusion/data/nyu_depth_v2_labeled.mat'
nyu_result_root = './results/NYU/'
eval_results_path = nyu_result_root + "eval_logs.csv"

save_folder = nyu_result_root

guidance_scale = 7.5
strength=0.5
num_inference_steps=20

# read mat file
f = h5py.File(nyu_path)

rgb_images = f['images']
depth_maps = f['depths']

"""
Nx4 matrix of accelerometer values indicated when each frame was taken. 
The columns contain the roll, yaw, pitch and tilt angle of the device.
"""
accel_data = f['accelData']

seg_mask = f['labels']

"""Cx1 cell array of the english names of each class."""
names = f['names']

""" map from english label names to class IDs (with C key-value pairs)"""
namesToIds = f['namesToIds']

"""Nx1 cell array of the name of the scene from which each image was taken."""
scenes = f['scenes']




C:\Users\User\AppData\Local\Temp\ipykernel_30008\393417447.py:12: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(nyu_path)


In [3]:
f[scenes[0][0]].attrs.keys()

<KeysViewHDF5 ['MATLAB_class', 'MATLAB_int_decode']>

In [6]:
#point_clouds.rebuild_point_clouds(rgb_image=rgb_images[i], depth_map=depth_maps[i],i=i, display=True, condition_type = "seg")

(PointCloud with 360448 points., PointCloud with 360448 points.)

In [8]:
#pcd = o3d.io.read_point_cloud( nyu_result_root + f"point_clouds/{i}_ground_pcd_from_seg.pcd")
#o3d.visualization.draw_geometries([pcd])

In [5]:
#pcd = o3d.io.read_point_cloud( nyu_result_root + f"point_clouds/{i}_gen_pcd_from_seg.pcd")
#o3d.visualization.draw_geometries([pcd])

In [6]:
i = 0

In [4]:
i = 0
original_pcd, generated_pcd = point_clouds.rebuild_point_clouds(rgb_images[i], depth_maps[i], i=i, result_root='./results/NYU/', display=False, condition_type="seg")
    

FileNotFoundError: [Errno 2] No such file or directory: './results/NYU/depth_maps/0_gen_depth_from_seg.npy'

In [ ]:
i = 1
original_pcd_2, generated_pcd_2 = point_clouds.rebuild_point_clouds(rgb_images[i], depth_maps[i], i=i, result_root='./results/NYU/', display=False, condition_type="seg")
    

In [ ]:
i = 2
original_pcd_3, generated_pcd_3 = point_clouds.rebuild_point_clouds(rgb_images[i], depth_maps[i], i=i, result_root='./results/NYU/', display=False, condition_type="seg")
    

In [18]:
roll = accel_data[0]
yaw = accel_data[1]
pitch = accel_data[2]
tilt = accel_data[3]


In [17]:
accel_data

<HDF5 dataset "accelData": shape (4, 1449), type "<f4">

In [46]:
import math

def rotation_matrix(pitch,roll,yaw):
    #taken from: https://www.zacobria.com/universal-robots-knowledge-base-tech-support-forum-hints-tips-cb2-cb3/index.php/python-code-example-of-converting-rpyeuler-angles-to-rotation-vectorangle-axis-for-universal-robots/
   
    yawMatrix = np.matrix([
    [math.cos(yaw), -math.sin(yaw), 0],
    [math.sin(yaw), math.cos(yaw), 0],
    [0, 0, 1]
    ])

    pitchMatrix = np.matrix([
    [math.cos(pitch), 0, math.sin(pitch)],
    [0, 1, 0],
    [-math.sin(pitch), 0, math.cos(pitch)]
    ])

    rollMatrix = np.matrix([
    [1, 0, 0],
    [0, math.cos(roll), -math.sin(roll)],
    [0, math.sin(roll), math.cos(roll)]
    ])

    R = yawMatrix * pitchMatrix * rollMatrix
    
    return R

In [47]:
R0 = rotation_matrix(pitch[0],roll[0],yaw[0])
R1 = rotation_matrix(pitch[1],roll[1],yaw[1])
R2 = rotation_matrix(pitch[2],roll[2],yaw[2])



rotated_original_pcd = original_pcd.rotate((R0))
rotated_original_pcd_2 = original_pcd_2.rotate((R1))
rotated_original_pcd_3 = original_pcd_3.rotate((R2))

In [83]:
def load_point_clouds(voxel_size=0.01):
    pcds_orig= []
    pcds_gen = []
    for i in range(5,8):
        original_pcd_i, generated_pcd_i = point_clouds.rebuild_point_clouds(rgb_images[i], depth_maps[i], i=i, result_root='./results/NYU/', display=False, condition_type="seg")
        pcd_down = original_pcd_i.voxel_down_sample(voxel_size=voxel_size)
        pcds_orig.append(pcd_down)
        
        pcd_down = generated_pcd_i.voxel_down_sample(voxel_size=voxel_size)
        pcds_gen.append(pcd_down)
    return pcds_orig, pcds_gen

In [84]:
voxel_size=0.001
pcds_orig, pcds_gen = load_point_clouds(voxel_size=voxel_size)
    
#pcds= [rotated_original_pcd, rotated_original_pcd_2, rotated_original_pcd_3]
#pcds_down = [pcd.voxel_down_sample(voxel_size=voxel_size) for pcd in [original_pcd, original_pcd_2, original_pcd_3]]

#o3d.visualization.draw_geometries(pcds)

In [86]:
o3d.visualization.draw_geometries(pcds_gen)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [87]:


def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [88]:
print("Full registration ...")
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_orig,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.3760, RMSE 0.0081
[Open3D DEBUG] Residual : 4.16e-05 (# of elements : 101251)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.3834, RMSE 0.0079
[Open3D DEBUG] Residual : 4.02e-05 (# of elements : 103264)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.3884, RMSE 0.0077
[Open3D DEBUG] Residual : 3.86e-05 (# of elements : 104613)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.3922, RMSE 0.0076
[Open3D DEBUG] Residual : 3.76e-05 (# of elements : 105624)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.3953, RMSE 0.0076
[Open3D DEBUG] Residual : 3.70e-05 (# of elements : 106471)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.3984, RMSE 0.0076
[Open3D DEBUG] Residual : 3.68e-05 (# of elements : 107308)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.4017, RMSE 0.0076
[Open3D DEBUG] Residual : 3.66e-05 (# of elements : 108193)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.4042, RMSE 0.0077
[Open3D DEBUG] Residual : 3.66e-05 (# of elem

[Open3D DEBUG] ICP Iteration #7: Fitness 0.0949, RMSE 0.0087
[Open3D DEBUG] Residual : 4.88e-05 (# of elements : 25562)
[Open3D DEBUG] ICP Iteration #8: Fitness 0.0951, RMSE 0.0086
[Open3D DEBUG] Residual : 4.90e-05 (# of elements : 25609)
[Open3D DEBUG] ICP Iteration #9: Fitness 0.0953, RMSE 0.0086
[Open3D DEBUG] Residual : 4.91e-05 (# of elements : 25664)
[Open3D DEBUG] ICP Iteration #10: Fitness 0.0953, RMSE 0.0085
[Open3D DEBUG] Residual : 4.87e-05 (# of elements : 25664)
[Open3D DEBUG] ICP Iteration #11: Fitness 0.0956, RMSE 0.0085
[Open3D DEBUG] Residual : 4.87e-05 (# of elements : 25747)
[Open3D DEBUG] ICP Iteration #12: Fitness 0.0960, RMSE 0.0085
[Open3D DEBUG] Residual : 4.86e-05 (# of elements : 25859)
[Open3D DEBUG] ICP Iteration #13: Fitness 0.0964, RMSE 0.0085
[Open3D DEBUG] Residual : 4.82e-05 (# of elements : 25960)
[Open3D DEBUG] ICP Iteration #14: Fitness 0.0969, RMSE 0.0085
[Open3D DEBUG] Residual : 4.84e-05 (# of elements : 26085)
[Open3D DEBUG] ICP Iteration #15: F

[Open3D DEBUG] ICP Iteration #7: Fitness 0.0061, RMSE 0.0010
[Open3D DEBUG] Residual : 4.48e-07 (# of elements : 1781)
[Open3D DEBUG] ICP Iteration #8: Fitness 0.0061, RMSE 0.0010
[Open3D DEBUG] Residual : 4.49e-07 (# of elements : 1778)
[Open3D DEBUG] ICP Iteration #9: Fitness 0.0061, RMSE 0.0010
[Open3D DEBUG] Residual : 4.53e-07 (# of elements : 1781)
[Open3D DEBUG] ICP Iteration #10: Fitness 0.0061, RMSE 0.0010
[Open3D DEBUG] Residual : 4.59e-07 (# of elements : 1782)
[Open3D DEBUG] ICP Iteration #11: Fitness 0.0060, RMSE 0.0010
[Open3D DEBUG] Residual : 4.51e-07 (# of elements : 1765)
[Open3D DEBUG] ICP Iteration #12: Fitness 0.0060, RMSE 0.0010
[Open3D DEBUG] Residual : 4.55e-07 (# of elements : 1767)
[Open3D DEBUG] ICP Iteration #13: Fitness 0.0060, RMSE 0.0010
[Open3D DEBUG] Residual : 4.52e-07 (# of elements : 1755)
[Open3D DEBUG] ICP Iteration #14: Fitness 0.0060, RMSE 0.0010
[Open3D DEBUG] Residual : 4.56e-07 (# of elements : 1760)
[Open3D DEBUG] ICP Iteration #15: Fitness 0

In [89]:
print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 3 nodes and 3 edges.
[Open3D DEBUG] Line process weight : 0.012880
[Open3D DEBUG] [Initial     ] residual : 1.919549e+01, lambda : 1.562100e-01
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] total time : 0.002 sec.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 3 nodes and 2 edges.
[Open3D DEBUG] Line process weight : 0.017574
[Open3D DEBUG] [Initial     ] residual : 1.688562e-11, lambda : 1.562100e-01
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] total time : 0.000 sec.
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


In [90]:
print("Transform points and display")
for point_id in range(len(pcds_orig)):
    print(pose_graph.nodes[point_id].pose)
    pcds_orig[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(pcds_orig)

Transform points and display
[[ 1.00000000e+00  5.62688652e-26  2.06795153e-25  1.03397577e-25]
 [ 5.62690104e-26  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.06795153e-25  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 0.99980545  0.01450927  0.01336176  0.00675335]
 [-0.01515285  0.99866433  0.04939573  0.02116762]
 [-0.01262722 -0.04958859  0.9986899   0.04096849]
 [ 0.          0.          0.          1.        ]]
[[ 0.89617027 -0.37467931 -0.2376852  -0.13754362]
 [ 0.34585059  0.92542889 -0.15481843 -0.01700631]
 [ 0.27796801  0.05654011  0.95892492  0.07660928]
 [ 0.          0.          0.          1.        ]]
